## Purpose: Train a model able to find matching fields in different documents

https://medium.com/@dhifallah.othmen/mesurer-la-similarit%C3%A9-s%C3%A9mantique-entre-phrases-avec-python-d08bf409200d
https://www.youtube.com/watch?v=ZeQgNhc0vag&ab_channel=KnowledgeCenter
https://www.youtube.com/playlist?list=PLhWB2ZsrULv-wEM8JDKA1zk8_2Lc88I-s

### 1 : Libraries import

In [14]:
import os
import nltk
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from datetime import datetime, date
from langdetect import detect
from joblib import dump, load
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import tree

### 2 : Definition of words pre-processing functions

In [15]:
def stem(list_text):
    """ Stemming """
    stemmer = nltk.stem.porter.PorterStemmer()
    return [stemmer.stem(token) for token in list_text]

def remove_punctuation(list_text):
    """ Removing punctuation """
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    return [token.translate(remove_punctuation_map) for token in list_text]

def delete_stop_words(list_text):
    """ Deleting not significant words """
    stop_words = stopwords.words('french') + stopwords.words('english')
    return [" ".join([word for word in token.split() if word not in stop_words]) for token in list_text]

def pre_process_tokenizer(list_text):
    """ Pre-processing by chaining transformations : stemming, removing punctuation, deleting stop words """
    return delete_stop_words(remove_punctuation(stem(list_text)))


### 3 : Building of a dictionnary containing, for french and english distinctively, training examples (key = df), corpus (key = corpus), base (key = base), vector projected in the base (key = X), label encoder (key = label_encoder), Y (key = Y)

In [16]:
df_csv_matching = pd.read_csv('matching_train.csv')
df_csv_matching['Language_client'] = df_csv_matching['column_client'].apply(detect)
df_csv_matching = df_csv_matching[['Language_client', 'column_client', 'column_daiteo']] # Reordering columns

In [17]:
dict_matching = {}

for language in ['en', 'fr']:
    dict_matching[language] = {}
    dict_matching[language]['df'] = df_csv_matching.loc[df_csv_matching['Language_client'] == language]
    dict_matching[language]['corpus'] = dict_matching[language]['df']['column_client'].tolist()
    dict_matching[language]['base'] = CountVectorizer()
    dict_matching[language]['base'].fit(pre_process_tokenizer(dict_matching[language]['corpus']))
    dict_matching[language]['X'] = pd.DataFrame(dict_matching[language]['base'].transform(pre_process_tokenizer(dict_matching[language]['df']['column_client'].tolist())).toarray(), columns = dict_matching[language]['base'].get_feature_names())
    dict_matching[language]['label_encoder'] = LabelEncoder()
    dict_matching[language]['label_encoder'].fit(dict_matching[language]['df']['column_daiteo'])
    dict_matching[language]['Y'] = pd.Series(dict_matching[language]['label_encoder'].transform(dict_matching[language]['df']['column_daiteo']))

In [18]:
dict_matching['en']['base'].get_feature_names()

['id', 'identifi', 'item', 'nam', 'numb', 'product']

In [19]:
dict_matching['fr']['base'].get_feature_names()

['coloris',
 'couleur',
 'dimens',
 'dimension',
 'dimensions',
 'masse',
 'numéro',
 'poids',
 'product',
 'produit',
 'référence',
 'teintes']

### 4 : Models training (Neural network), for french and english distinctively

In [116]:
for language in ['en', 'fr']:
    
    ## Selecting appropriate training examples & labels
    X = dict_matching[language]['X']
    Y = dict_matching[language]['Y']
    
    ## Definition of input & output shapes of neural network
    input_shape_nn = len(dict_matching[language]['base'].get_feature_names())
    output_shape_nn = len(dict_matching[language]['label_encoder'].classes_)
    
    ## Definition of neural network layers' shapes
    dict_matching[language]['model'] = tf.keras.models.Sequential([
        keras.layers.Dense(32, input_dim = input_shape_nn, activation = tf.nn.relu),
        keras.layers.Dense(output_shape_nn, activation = tf.nn.softmax),
    ])
    
    ## Training of neural network
    dict_matching[language]['model'].compile(optimizer = 'SGD', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    dict_matching[language]['model'].fit(X, Y, epochs = 600)
    

Epoch 1/600
1/1 [==============================] - 0s 6ms/step - loss: 0.8747 - accuracy: 0.3333
Epoch 2/600
1/1 [==============================] - 0s 2ms/step - loss: 0.8643 - accuracy: 0.3333
Epoch 3/600
1/1 [==============================] - 0s 2ms/step - loss: 0.8543 - accuracy: 0.3333
Epoch 4/600
1/1 [==============================] - 0s 6ms/step - loss: 0.8450 - accuracy: 0.3333
Epoch 5/600
1/1 [==============================] - 0s 2ms/step - loss: 0.8358 - accuracy: 0.3333
Epoch 6/600
1/1 [==============================] - 0s 2ms/step - loss: 0.8269 - accuracy: 0.3333
Epoch 7/600
1/1 [==============================] - 0s 5ms/step - loss: 0.8184 - accuracy: 0.3333
Epoch 8/600
1/1 [==============================] - 0s 3ms/step - loss: 0.8100 - accuracy: 0.3333
Epoch 9/600
1/1 [==============================] - 0s 6ms/step - loss: 0.8018 - accuracy: 0.3333
Epoch 10/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7936 - accuracy: 0.3333
Epoch 11/600
1/1 [===========

1/1 [==============================] - 0s 2ms/step - loss: 0.4645 - accuracy: 0.8333
Epoch 85/600
1/1 [==============================] - 0s 4ms/step - loss: 0.4622 - accuracy: 0.8333
Epoch 86/600
1/1 [==============================] - 0s 999us/step - loss: 0.4598 - accuracy: 0.8333
Epoch 87/600
1/1 [==============================] - 0s 2ms/step - loss: 0.4575 - accuracy: 0.8333
Epoch 88/600
1/1 [==============================] - 0s 2ms/step - loss: 0.4552 - accuracy: 0.8333
Epoch 89/600
1/1 [==============================] - 0s 998us/step - loss: 0.4529 - accuracy: 0.8333
Epoch 90/600
1/1 [==============================] - 0s 2ms/step - loss: 0.4507 - accuracy: 0.8333
Epoch 91/600
1/1 [==============================] - 0s 5ms/step - loss: 0.4484 - accuracy: 0.8333
Epoch 92/600
1/1 [==============================] - 0s 4ms/step - loss: 0.4463 - accuracy: 0.8333
Epoch 93/600
1/1 [==============================] - 0s 3ms/step - loss: 0.4441 - accuracy: 0.8333
Epoch 94/600
1/1 [===========

1/1 [==============================] - 0s 4ms/step - loss: 0.3289 - accuracy: 0.8333
Epoch 167/600
1/1 [==============================] - 0s 2ms/step - loss: 0.3277 - accuracy: 0.8333
Epoch 168/600
1/1 [==============================] - 0s 4ms/step - loss: 0.3266 - accuracy: 0.8333
Epoch 169/600
1/1 [==============================] - 0s 4ms/step - loss: 0.3254 - accuracy: 0.8333
Epoch 170/600
1/1 [==============================] - 0s 2ms/step - loss: 0.3243 - accuracy: 0.8333
Epoch 171/600
1/1 [==============================] - 0s 2ms/step - loss: 0.3232 - accuracy: 0.8333
Epoch 172/600
1/1 [==============================] - 0s 4ms/step - loss: 0.3220 - accuracy: 0.8333
Epoch 173/600
1/1 [==============================] - 0s 5ms/step - loss: 0.3209 - accuracy: 0.8333
Epoch 174/600
1/1 [==============================] - 0s 4ms/step - loss: 0.3198 - accuracy: 0.8333
Epoch 175/600
1/1 [==============================] - 0s 999us/step - loss: 0.3187 - accuracy: 0.8333
Epoch 176/600
1/1 [===

1/1 [==============================] - 0s 2ms/step - loss: 0.2514 - accuracy: 1.0000
Epoch 249/600
1/1 [==============================] - 0s 3ms/step - loss: 0.2506 - accuracy: 1.0000
Epoch 250/600
1/1 [==============================] - 0s 2ms/step - loss: 0.2498 - accuracy: 1.0000
Epoch 251/600
1/1 [==============================] - 0s 2ms/step - loss: 0.2491 - accuracy: 1.0000
Epoch 252/600
1/1 [==============================] - 0s 2ms/step - loss: 0.2483 - accuracy: 1.0000
Epoch 253/600
1/1 [==============================] - 0s 3ms/step - loss: 0.2475 - accuracy: 1.0000
Epoch 254/600
1/1 [==============================] - 0s 2ms/step - loss: 0.2467 - accuracy: 1.0000
Epoch 255/600
1/1 [==============================] - 0s 2ms/step - loss: 0.2460 - accuracy: 1.0000
Epoch 256/600
1/1 [==============================] - 0s 2ms/step - loss: 0.2452 - accuracy: 1.0000
Epoch 257/600
1/1 [==============================] - 0s 2ms/step - loss: 0.2444 - accuracy: 1.0000
Epoch 258/600
1/1 [=====

1/1 [==============================] - 0s 4ms/step - loss: 0.1963 - accuracy: 1.0000
Epoch 330/600
1/1 [==============================] - 0s 2ms/step - loss: 0.1957 - accuracy: 1.0000
Epoch 331/600
1/1 [==============================] - 0s 2ms/step - loss: 0.1952 - accuracy: 1.0000
Epoch 332/600
1/1 [==============================] - ETA: 0s - loss: 0.1946 - accuracy: 1.00 - 0s 3ms/step - loss: 0.1946 - accuracy: 1.0000
Epoch 333/600
1/1 [==============================] - 0s 2ms/step - loss: 0.1940 - accuracy: 1.0000
Epoch 334/600
1/1 [==============================] - 0s 2ms/step - loss: 0.1934 - accuracy: 1.0000
Epoch 335/600
1/1 [==============================] - 0s 2ms/step - loss: 0.1928 - accuracy: 1.0000
Epoch 336/600
1/1 [==============================] - 0s 2ms/step - loss: 0.1923 - accuracy: 1.0000
Epoch 337/600
1/1 [==============================] - 0s 3ms/step - loss: 0.1917 - accuracy: 1.0000
Epoch 338/600
1/1 [==============================] - 0s 2ms/step - loss: 0.1911 -

...

1/1 [==============================] - 0s 2ms/step - loss: 1.1259 - accuracy: 0.7273
Epoch 222/600
1/1 [==============================] - 0s 3ms/step - loss: 1.1247 - accuracy: 0.7273
Epoch 223/600
1/1 [==============================] - 0s 3ms/step - loss: 1.1235 - accuracy: 0.7273
Epoch 224/600
1/1 [==============================] - 0s 3ms/step - loss: 1.1223 - accuracy: 0.7273
Epoch 225/600
1/1 [==============================] - 0s 3ms/step - loss: 1.1211 - accuracy: 0.7273
Epoch 226/600
1/1 [==============================] - 0s 3ms/step - loss: 1.1198 - accuracy: 0.7273
Epoch 227/600
1/1 [==============================] - 0s 5ms/step - loss: 1.1187 - accuracy: 0.7273
Epoch 228/600
1/1 [==============================] - 0s 2ms/step - loss: 1.1174 - accuracy: 0.7273
Epoch 229/600
1/1 [==============================] - 0s 2ms/step - loss: 1.1162 - accuracy: 0.7273
Epoch 230/600
1/1 [==============================] - 0s 2ms/step - loss: 1.1151 - accuracy: 0.7273
Epoch 231/600
1/1 [=====

1/1 [==============================] - 0s 6ms/step - loss: 1.0302 - accuracy: 0.7273
Epoch 303/600
1/1 [==============================] - 0s 2ms/step - loss: 1.0291 - accuracy: 0.7273
Epoch 304/600
1/1 [==============================] - 0s 5ms/step - loss: 1.0279 - accuracy: 0.7273
Epoch 305/600
1/1 [==============================] - 0s 2ms/step - loss: 1.0268 - accuracy: 0.7273
Epoch 306/600
1/1 [==============================] - 0s 2ms/step - loss: 1.0256 - accuracy: 0.7273
Epoch 307/600
1/1 [==============================] - 0s 4ms/step - loss: 1.0245 - accuracy: 0.7273
Epoch 308/600
1/1 [==============================] - 0s 2ms/step - loss: 1.0233 - accuracy: 0.7273
Epoch 309/600
1/1 [==============================] - 0s 2ms/step - loss: 1.0222 - accuracy: 0.7273
Epoch 310/600
1/1 [==============================] - 0s 2ms/step - loss: 1.0210 - accuracy: 0.7273
Epoch 311/600
1/1 [==============================] - 0s 3ms/step - loss: 1.0199 - accuracy: 0.7273
Epoch 312/600
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.9350 - accuracy: 0.8182
Epoch 385/600
1/1 [==============================] - ETA: 0s - loss: 0.9338 - accuracy: 0.81 - 0s 4ms/step - loss: 0.9338 - accuracy: 0.8182
Epoch 386/600
1/1 [==============================] - 0s 3ms/step - loss: 0.9326 - accuracy: 0.8182
Epoch 387/600
1/1 [==============================] - 0s 3ms/step - loss: 0.9315 - accuracy: 0.8182
Epoch 388/600
1/1 [==============================] - 0s 3ms/step - loss: 0.9303 - accuracy: 0.8182
Epoch 389/600
1/1 [==============================] - 0s 6ms/step - loss: 0.9291 - accuracy: 0.8182
Epoch 390/600
1/1 [==============================] - ETA: 0s - loss: 0.9280 - accuracy: 0.81 - 0s 3ms/step - loss: 0.9280 - accuracy: 0.8182
Epoch 391/600
1/1 [==============================] - 0s 4ms/step - loss: 0.9268 - accuracy: 0.8182
Epoch 392/600
1/1 [==============================] - 0s 3ms/step - loss: 0.9257 - accuracy: 0.8182
Epoch 393/600
1/1 [====================

1/1 [==============================] - 0s 2ms/step - loss: 0.8446 - accuracy: 0.8182
Epoch 466/600
1/1 [==============================] - 0s 4ms/step - loss: 0.8435 - accuracy: 0.8182
Epoch 467/600
1/1 [==============================] - 0s 997us/step - loss: 0.8425 - accuracy: 0.8182
Epoch 468/600
1/1 [==============================] - 0s 2ms/step - loss: 0.8414 - accuracy: 0.8182
Epoch 469/600
1/1 [==============================] - 0s 5ms/step - loss: 0.8403 - accuracy: 0.8182
Epoch 470/600
1/1 [==============================] - 0s 6ms/step - loss: 0.8392 - accuracy: 0.8182
Epoch 471/600
1/1 [==============================] - 0s 2ms/step - loss: 0.8381 - accuracy: 0.8182
Epoch 472/600
1/1 [==============================] - 0s 4ms/step - loss: 0.8370 - accuracy: 0.8182
Epoch 473/600
1/1 [==============================] - 0s 5ms/step - loss: 0.8359 - accuracy: 0.8182
Epoch 474/600
1/1 [==============================] - 0s 6ms/step - loss: 0.8349 - accuracy: 0.8182
Epoch 475/600
1/1 [===

1/1 [==============================] - 0s 2ms/step - loss: 0.7577 - accuracy: 0.9091
Epoch 547/600
1/1 [==============================] - 0s 4ms/step - loss: 0.7567 - accuracy: 0.9091
Epoch 548/600
1/1 [==============================] - 0s 4ms/step - loss: 0.7556 - accuracy: 0.9091
Epoch 549/600
1/1 [==============================] - 0s 3ms/step - loss: 0.7546 - accuracy: 0.9091
Epoch 550/600
1/1 [==============================] - 0s 3ms/step - loss: 0.7536 - accuracy: 0.9091
Epoch 551/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7525 - accuracy: 0.9091
Epoch 552/600
1/1 [==============================] - 0s 4ms/step - loss: 0.7515 - accuracy: 0.9091
Epoch 553/600
1/1 [==============================] - 0s 3ms/step - loss: 0.7505 - accuracy: 0.9091
Epoch 554/600
1/1 [==============================] - 0s 3ms/step - loss: 0.7495 - accuracy: 0.9091
Epoch 555/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7484 - accuracy: 0.9091
Epoch 556/600
1/1 [=====

In [117]:
dict_matching['en']['model'].summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 32)                224       
_________________________________________________________________
dense_49 (Dense)             (None, 2)                 66        
Total params: 290
Trainable params: 290
Non-trainable params: 0
_________________________________________________________________


In [118]:
dict_matching['fr']['model'].summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 32)                416       
_________________________________________________________________
dense_51 (Dense)             (None, 5)                 165       
Total params: 581
Trainable params: 581
Non-trainable params: 0
_________________________________________________________________


### 5 : Models saving

In [119]:
for language in ['en', 'fr']:
    dict_matching[language]['model'].save('.\save_13-10-2020_' + language + '_nn')

INFO:tensorflow:Assets written to: .\save_13-10-2020_en_nn\assets
INFO:tensorflow:Assets written to: .\save_13-10-2020_fr_nn\assets


### 6 : Models loading

In [120]:
model = keras.models.load_model('.\save_13-10-2020_fr_nn')
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 32)                416       
_________________________________________________________________
dense_51 (Dense)             (None, 5)                 165       
Total params: 581
Trainable params: 581
Non-trainable params: 0
_________________________________________________________________


### 7 : Model prediction

In [121]:
le = dict_matching['fr']['label_encoder']
x = dict_matching['fr']['X']

le.inverse_transform(list(np.argmax(model.predict(x), axis = 1)))


array(['Product color', 'Product reference', 'Product dimension',
       'Product dimension', 'Product dimension', 'Product weight',
       'Product weight', 'Product color', 'Product color',
       'Product color', 'Product color'], dtype=object)

In [122]:
dict_matching['fr']['df']

,Language_client,column_client,column_daiteo,Predicted_class
0,fr,produit,Product name,Product color
5,fr,Numéro référence produit,Product reference,Product color
17,fr,Product dimension,Product dimension,Product color
18,fr,Dimension du produit,Product dimension,Product dimension
19,fr,Dimensions produit,Product dimension,Product dimension
25,fr,poids produit,Product weight,Product color
26,fr,masse produit,Product weight,Product weight
27,fr,couleur produit,Product color,Product color
28,fr,couleur,Product color,Product color
34,fr,teintes produits,Product color,Product color
